<a href="https://colab.research.google.com/github/kalyani234/Drug_Project/blob/main/drug_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# coding=utf-8
import sqlite3
import requests
from bs4 import BeautifulSoup as bs
import math
import pandas as pd
import time

# Establish database connection
conn = sqlite3.connect("Drug.db")
cur = conn.cursor()

def create_tables():
    """Create necessary tables in the SQLite database."""
    # Create the 'event' table
    cur.execute("""
        CREATE TABLE IF NOT EXISTS event (
            id1 TEXT,
            name1 TEXT,
            id2 TEXT,
            name2 TEXT,
            interaction TEXT
        )
    """)

    # Create the 'drug' table
    cur.execute("""
        CREATE TABLE IF NOT EXISTS drug (
            id TEXT PRIMARY KEY,
            name TEXT,
            interaction TEXT,
            smile TEXT,
            target TEXT,
            enzyme TEXT,
            carrier TEXT,
            transporter TEXT
        )
    """)

    conn.commit()

def download_page(url, num_retries=150):
    """Download HTML content from the given URL with retries on failure."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        return bs(response.content, "html.parser")
    except requests.RequestException as e:
        if num_retries > 0:
            print(f"Retrying... ({num_retries} attempts left)")
            time.sleep(1)
            return download_page(url, num_retries - 1)
        print("Download failed after retries:", e)
        return None

def get_drug_ids(smd_pages, bd_pages):
    """Retrieve drug IDs for Small Molecule and Biotech drugs from DrugBank."""
    url_ids = []
    urls = {
        "smd": "https://www.drugbank.ca/drugs?approved=1&c=name&d=up&experimental=1&page=",
        "bd": "https://www.drugbank.ca/biotech_drugs?approved=1&page="
    }

    for drug_type, pages in [("smd", smd_pages), ("bd", bd_pages)]:
        for page in range(1, pages + 1):
            print(f"Fetching {drug_type} page {page}")
            soup = download_page(urls[drug_type] + str(page))
            if soup:
                url_ids += [a['href'].split('/')[-1] for a in soup.select('.name-value strong a')]
    return url_ids

def parse_identification(soup):
    """Extract identification details from the soup object."""
    iden = {dt.text: dd.text for dt, dd in zip(soup.select("dl dt"), soup.select("dl dd"))}
    return iden

def fetch_interactions(url_id, drug_name):
    """Retrieve interactions for a given drug."""
    interactions = []
    url = f'https://www.drugbank.ca/drugs/{url_id}/drug_interactions.json?'
    try:
        total_records = requests.get(url).json().get('recordsTotal', 0)
        for i in range(math.ceil(total_records / 100)):
            new_url = f"{url}start={i * 100}&length=100"
            for interaction_data in requests.get(new_url).json().get('data', []):
                interaction_id = bs(interaction_data[0], "lxml").find("a")["href"].split("/")[-1]
                interaction_name = interaction_data[0].split('>')[1]
                interaction_desc = interaction_data[1]
                interactions.append((interaction_id, interaction_name, interaction_desc))
                cur.execute("INSERT INTO event (id1, name1, id2, name2, interaction) VALUES (?, ?, ?, ?, ?)",
                            (url_id, drug_name, interaction_id, interaction_name, interaction_desc))
        return interactions
    except Exception as e:
        print(f"Failed to fetch interactions for {url_id}: {e}")
        return []

def extract_attributes(soup):
    """Extract drug attributes like targets, enzymes, carriers, and transporters."""
    attributes = {}
    try:
        for container in soup.select('.bond-list-container'):
            attribute_values = '|'.join(j['href'].split('/')[-1] for j in container.select('.bond-list strong a'))
            attributes[container.h3.text] = attribute_values
    except Exception as e:
        print("Failed to extract attributes:", e)
    return attributes

In [2]:
def main():
    # Create necessary tables
    create_tables()

    # Load drug IDs from the Excel file
    drug_list = pd.read_excel("/content/drug_list.xlsx", header=None)
    url_ids = drug_list.iloc[:, 0]

    basic_url = "https://www.drugbank.ca/drugs/"
    for drug_id in url_ids:
        soup = download_page(basic_url + str(drug_id))
        if not soup:
            continue

        drug_data = parse_identification(soup)
        drug_name = drug_data.get("Name", "")
        drug_smile = drug_data.get("SMILES", "") if drug_data.get("SMILES") != "Not Available" else ""
        interactions = fetch_interactions(drug_id, drug_name)

        attributes = extract_attributes(soup)
        drug_targets = attributes.get("Targets", "")
        drug_enzymes = attributes.get("Enzymes", "")
        drug_carriers = attributes.get("Carriers", "")
        drug_transporters = attributes.get("Transporters", "")

        cur.execute("""
            INSERT INTO drug (id, name, interaction, smile, target, enzyme, carrier, transporter)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """, (drug_id, drug_name, '|'.join(i[0] for i in interactions), drug_smile,
                  drug_targets, drug_enzymes, drug_carriers, drug_transporters))
    conn.commit()
    conn.close()

if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
Failed to fetch interactions for DB06251: Expecting value: line 1 column 1 (char 0)
Failed to fetch interactions for DB06255: Expecting value: line 1 column 1 (char 0)
Failed to fetch interactions for DB06258: Expecting value: line 1 column 1 (char 0)
Failed to fetch interactions for DB06261: Expecting value: line 1 column 1 (char 0)
Failed to fetch interactions for DB06262: Expecting value: line 1 column 1 (char 0)
Failed to fetch interactions for DB06263: Expecting value: line 1 column 1 (char 0)
Failed to fetch interactions for DB06264: Expecting value: line 1 column 1 (char 0)
Failed to fetch interactions for DB06266: Expecting value: line 1 column 1 (char 0)
Failed to fetch interactions for DB06267: Expecting value: line 1 column 1 (char 0)
Failed to fetch interactions for DB06268: Expecting value: line 1 column 1 (char 0)
Failed to fetch interactions for DB06271: Expecting value: line 1 column 1 (char 0)
Failed to fetch interacti

In [3]:
import sqlite3
import pandas as pd

# Reconnect to the database
conn = sqlite3.connect("Drug.db")

# Query the drug table to retrieve all data
df = pd.read_sql_query("SELECT * FROM drug", conn)

# Save the data to a CSV file
df.to_csv("drug_data.csv", index=False, encoding='utf-8')

# Close the database connection
conn.close()

print("Data successfully exported to drug_data1.csv")

Data successfully exported to drug_data1.csv
